## Importing relevant libraries

In [40]:
from __future__ import annotations

import logging
import warnings
from time import time
from collections.abc import Callable, Sequence

import numpy as np

from qiskit_algorithms.gradients import BaseEstimatorGradient
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BaseEstimator, Sampler, Estimator
from qiskit.quantum_info.operators.base_operator import BaseOperator
from qiskit.quantum_info import Statevector
from qiskit_algorithms.utils import algorithm_globals

from qiskit_algorithms.exceptions import AlgorithmError
from qiskit_algorithms.list_or_dict import ListOrDict
from qiskit_algorithms.optimizers import Optimizer, Minimizer, OptimizerResult
from qiskit_algorithms.variational_algorithm import (
    VariationalAlgorithm,
    VariationalResult,
)
from qiskit_algorithms.eigensolvers import Eigensolver, EigensolverResult

from qiskit_algorithms.observables_evaluator import estimate_observables

import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm, null_space
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Import optimizers
# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

## Instantiating the SSVQE class (from Qiskit)

In [44]:
logger = logging.getLogger(__name__)


class SSVQE(VariationalAlgorithm, Eigensolver):
    r"""The Subspace Search Variational Quantum Eigensolver algorithm.
    `SSVQE <https://arxiv.org/abs/1810.09434>`__ is a hybrid quantum-classical
    algorithm that uses a variational technique to find the low-lying eigenvalues
    of the Hamiltonian :math:`H` of a given system. SSVQE can be seen as
    a natural generalization of VQE. Whereas VQE minimizes the expectation
    value of :math:`H` with respect to the ansatz state, SSVQE takes a set
    of mutually orthogonal input states :math:`\{| \psi_{i}} \rangle\}_{i=0}^{k-1}`,
    applies the same parameterized ansatz circuit :math:`U(\vec\theta)` to all of them,
    then minimizes a weighted sum of the expectation values of :math:`H` with respect
    to these states.

    An instance of SSVQE requires defining four algorithmic sub-components:

    An :attr:`estimator` to compute the expectation values of operators, an integer ``k`` denoting
    the number of eigenstates that the algorithm will attempt to find, an ansatz which is a
    :class:`QuantumCircuit`, and one of the classical :mod:`~qiskit.algorithms.optimizers`.

    The ansatz is varied, via its set of parameters, by the optimizer, such that it works towards
    a set of mutually orthogonal states, as determined by the parameters applied to the ansatz,
    that will result in the minimum weighted sum of expectation values being measured
    of the input operator (Hamiltonian) with respect to these states. The weights given
    to this list of expectation values is given by ``weight_vector``. An optional array of
    parameter values, via the ``initial_point``, may be provided as the starting point for
    the search of the low-lying eigenvalues. This feature is particularly useful such as
    when there are reasons to believe that the solution point is close to a particular
    point. The length of the ``initial_point`` list value must match the number of the
    parameters expected by the ansatz being used. If the ``initial_point`` is left at the
    default of ``None``, then SSVQE will look to the ansatz for a preferred value, based
    on its given initial state. If the ansatz returns ``None``, then a random point will
    be generated within the parameter bounds set, as per above. If the ansatz provides
    ``None`` as the lower bound, then SSVQE will default it to :math:`-2\pi`; similarly,
    if the ansatz returns ``None`` as the upper bound, the default value will be :math:`2\pi`.

    An optional list of initial states, via the ``initial_states``, may also be provided.
    Choosing these states appropriately is a critical part of the algorithm. They must
    be mutually orthogonal because this is how the algorithm enforces the mutual
    orthogonality of the solution states. If the ``initial_states`` is left as ``None``,
    then SSVQE will automatically generate a list of computational basis states and use
    these as the initial states. For many physically-motivated problems, it is advised
    to not rely on these default values as doing so can easily result in an unphysical
    solution being returned. For example, if one wishes to find the low-lying excited
    states of a molecular Hamiltonian, then we expect the output states to belong to a
    particular particle-number subspace. If an ansatz that preserves particle number
    such as :class:`UCCSD` is used, then states belonging to the incorrect particle
    number subspace will be returned if the ``initial_states`` are not in the correct
    particle number subspace. A similar statement can often be made for the
    spin-magnetization quantum number.

    A minimal example of how one may initialize an instance of ``SSVQE`` and use it
    to compute the low-lying eigenvalues of an operator:

    .. code-block:: python

      from qiskit import QuantumCircuit
      from qiskit.quantum_info import Pauli
      from qiskit.primitives import Estimator
      from qiskit.circuit.library import RealAmplitudes
      from qiskit.algorithms.optimizers import SPSA
      from qiskit.algorithms.eigensolvers import SSVQE

      operator = Pauli("ZZ")
      input_states = [QuantumCircuit(2), QuantumCircuit(2)]
      input_states[0].x(0)
      input_states[1].x(1)

      ssvqe_instance = SSVQE(k=2,
                            estimator=Estimator(),
                            optimizer=SPSA(),
                            ansatz=RealAmplitudes(2),
                            initial_states=input_states)

      result = ssvqe_instance.compute_eigenvalues(operator)

    The following attributes can be set via the initializer but can also be read and
    updated once the SSVQE object has been constructed.

    Attributes:
            estimator (BaseEstimator): The primitive instance used to perform the expectation
                estimation of observables.
            k (int): The number of eigenstates that SSVQE will attempt to find.
            ansatz (QuantumCircuit): A parameterized circuit used as an ansatz for the
                wave function.
            optimizer (Optimizer): A classical optimizer, which can be either a Qiskit optimizer
                or a callable that takes an array as input and returns a Qiskit or SciPy
                optimization result.
            gradient (BaseEstimatorGradient | None): An optional estimator gradient to be used
                with the optimizer.
            initial_states (Sequence[QuantumCircuit]): An optional list of mutually orthogonal
                initial states. If ``None``, then SSVQE will set these to be a list of mutually
                orthogonal computational basis states.
            weight_vector (Sequence[float]): A 1D array of real positive-valued numbers to assign
                as weights to each of the expectation values. If ``None``, then SSVQE will default
                to [k, k-1, ..., 1].
            callback (Callable[[int, np.ndarray, Sequence[float], dict[str, Any]], None] | None): A
                function that can access the intermediate data at each optimization step.
                These data are the evaluation count, the optimizer parameters for
                the ansatz, the evaluated mean energies, and the metadata dictionary.
            check_input_states_orthogonality: A boolean that sets whether or not to check
                that the value of initial_states passed consists of a mutually orthogonal
                set of states. If ``True``, then SSVQE will check that these states are mutually
                orthogonal and return an :class:`AlgorithmError` if they are not.
                This is set to ``True`` by default, but setting this to ``False`` may be desirable
                for larger numbers of qubits to avoid exponentially large computational overhead.
    """

    def __init__(
        self,
        estimator: BaseEstimator,
        k: int | None = 2,
        ansatz: QuantumCircuit | None = None,
        optimizer: Optimizer | Minimizer | None = None,
        initial_point: Sequence[float] | None = None,
        initial_states: list[QuantumCircuit] | None = None,
        weight_vector: Sequence[float] | Sequence[int] | None = None,
        gradient: BaseEstimatorGradient | None = None,
        callback: Callable[[int, np.ndarray, Sequence[float], float], None]
        | None = None,
        check_input_states_orthogonality: bool = True,
    ) -> None:
        """
        Args:
            estimator: The estimator primitive.
            k: The number of eigenstates that the algorithm will attempt to find.
            ansatz: A parameterized circuit used as Ansatz for the wave function.
            optimizer: A classical optimizer. Can either be a Qiskit optimizer or a callable
                that takes an array as input and returns a Qiskit or SciPy optimization result.
            initial_point: An optional initial point (i.e. initial parameter values)
                for the optimizer. If ``None`` then VQE will look to the ansatz for a preferred
                point and if not will simply compute a random one.
            initial_states: An optional list of mutually orthogonal initial states.
                If ``None``, then SSVQE will set these to be a list of mutually orthogonal
                computational basis states.
            weight_vector: An optional list or array of real positive numbers with length
                equal to the value of ``num_states`` to be used in the weighted energy summation
                objective function. This fixes the ordering of the returned eigenstate/eigenvalue
                pairs. If ``None``, then SSVQE will default to [n, n-1, ..., 1] for `k` = n.
            gradient: An optional gradient function or operator for optimizer.
            callback: A callback that can access the intermediate data at each optimization step.
                These data are: the evaluation count, the optimizer ansatz parameters,
                the evaluated mean energies, and the metadata dictionary.
            check_input_states_orthogonality: A boolean that sets whether or not to check
                that the value of ``initial_states`` passed consists of a mutually orthogonal
                set of states. If ``True``, then SSVQE will check that these states are mutually
                orthogonal and return an error if they are not. This is set to ``True`` by default,
                but setting this to ``False`` may be desirable for larger numbers of qubits to avoid
                exponentially large computational overhead before the simulation even starts.
        """

        super().__init__()

        self.k = k
        self.initial_states = initial_states
        self.weight_vector = weight_vector
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.initial_point = initial_point
        self.gradient = gradient
        self.callback = callback
        self.estimator = estimator
        self.check_initial_states_orthogonal = check_input_states_orthogonality

    @property
    def initial_point(self) -> Sequence[float] | None:
        """Returns initial point"""
        return self._initial_point

    @initial_point.setter
    def initial_point(self, initial_point: Sequence[float] | None):
        """Sets initial point"""
        self._initial_point = initial_point

    @classmethod
    def supports_aux_operators(cls) -> bool:
        return True

    def compute_eigenvalues(
        self,
        operator: BaseOperator | SparsePauliOp,
        aux_operators: ListOrDict[BaseOperator | SparsePauliOp] | None = None,
    ) -> EigensolverResult:

        ansatz = self._check_operator_ansatz(operator)

        initial_point = _validate_initial_point(self.initial_point, ansatz)

        initial_states = self._check_operator_initial_states(
            self.initial_states, operator
        )

        bounds = _validate_bounds(ansatz)

        initialized_ansatz_list = [
            initial_states[n].compose(ansatz) for n in range(self.k)
        ]

        self.weight_vector = self._check_weight_vector(self.weight_vector)

        evaluate_weighted_energy_sum = self._get_evaluate_weighted_energy_sum(
            initialized_ansatz_list, operator
        )

        if self.gradient is not None:  # need to implement _get_evaluate_gradient
            evaluate_gradient = self._get_evalute_gradient(
                initialized_ansatz_list, operator
            )
        else:
            evaluate_gradient = None

        if aux_operators:
            zero_op = SparsePauliOp.from_list([("I" * self.ansatz.num_qubits, 0)])

            # Convert the None and zero values when aux_operators is a list.
            # Drop None and convert zero values when aux_operators is a dict.
            if isinstance(aux_operators, list):
                key_op_iterator = enumerate(aux_operators)
                converted = [zero_op] * len(aux_operators)
            else:
                key_op_iterator = aux_operators.items()
                converted = {}
            for key, op in key_op_iterator:
                if op is not None:
                    converted[key] = zero_op if op == 0 else op

            aux_operators = converted

        else:
            aux_operators = None

        start_time = time()

        if callable(self.optimizer):
            optimizer_result = self.optimizer(
                fun=evaluate_weighted_energy_sum,
                x0=initial_point,
                jac=evaluate_gradient,
                bounds=bounds,
            )
        else:
            optimizer_result = self.optimizer.minimize(
                fun=evaluate_weighted_energy_sum,
                x0=initial_point,
                jac=evaluate_gradient,
                bounds=bounds,
            )

        optimizer_time = time() - start_time

        logger.info(
            "Optimization complete in %s seconds.\nFound opt_params %s",
            optimizer_time,
            optimizer_result.x,
        )

        if aux_operators is not None:
            bound_ansatz_list = [
                initialized_ansatz_list[n].bind_parameters(optimizer_result.x)
                for n in range(self.k)
            ]

            aux_values_list = [
                estimate_observables(
                    self.estimator,
                    bound_ansatz_list[n],
                    aux_operators,
                )
                for n in range(self.k)
            ]
        else:
            aux_values_list = None

        return self._build_ssvqe_result(
            optimizer_result,
            aux_values_list,
            optimizer_time,
            operator,
            initialized_ansatz_list,
        )

    def _get_evaluate_weighted_energy_sum(
        self,
        initialized_ansatz_list: list[QuantumCircuit],
        operator: BaseOperator | SparsePauliOp,
    ) -> tuple[Callable[[np.ndarray], float | list[float]], dict]:
        """Returns a function handle to evaluate the weighted energy sum at given parameters
        for the ansatz. This is the objective function to be passed to the optimizer
        that is used for evaluation.
        Args:
            initialized_anastz_list: A list consisting of copies of the ansatz initialized
                in the initial states.
            operator: The operator whose expectation value with respect to each of the
                states in ``initialzed_ansatz_list`` is being measured.
        Returns:
            Weighted expectation value sum of the operator for each parameter.
        Raises:
            AlgorithmError: If the primitive job to evaluate the weighted energy
                sum fails.
        """
        num_parameters = initialized_ansatz_list[0].num_parameters

        eval_count = 0

        def evaluate_weighted_energy_sum(parameters):
            nonlocal eval_count
            parameters = np.reshape(parameters, (-1, num_parameters)).tolist()
            batchsize = len(parameters)

            try:
                job = self.estimator.run(
                    [
                        initialized_ansatz_list[m]
                        for n in range(batchsize)
                        for m in range(self.k)
                    ],
                    [operator] * self.k * batchsize,
                    [parameters[n] for n in range(batchsize) for m in range(self.k)],
                )
                result = job.result()
                values = result.values

                energies = np.reshape(values, (batchsize, self.k))
                weighted_energy_sums = np.dot(energies, self.weight_vector).tolist()
                energies = energies.tolist()

            except Exception as exc:
                raise AlgorithmError(
                    "The primitive job to evaluate the energy failed!"
                ) from exc

            if self.callback is not None:
                metadata = result.metadata
                for params, energies_value, metadata in zip(
                    parameters, energies, metadata
                ):
                    eval_count += 1
                    self.callback(eval_count, params, energies_value, metadata)

            return (
                weighted_energy_sums[0]
                if len(weighted_energy_sums) == 1
                else weighted_energy_sums
            )

        return evaluate_weighted_energy_sum

    def _get_evalute_gradient(
        self,
        initialized_ansatz_list: list[QuantumCircuit],
        operator: BaseOperator | SparsePauliOp,
    ) -> tuple[Callable[[np.ndarray], np.ndarray]]:
        """Get a function handle to evaluate the gradient at given parameters for the ansatz.
        Args:
            initialized_ansatz_list: The list of initialized ansatz preparing the quantum states.
            operator: The operator whose energy to evaluate.
        Returns:
            A function handle to evaluate the gradient at given parameters for the initialized
            ansatz list.
        Raises:
            AlgorithmError: If the primitive job to evaluate the gradient fails.
        """

        def evaluate_gradient(parameters):
            # broadcasting not required for the estimator gradients
            try:
                job = self.gradient.run(
                    initialized_ansatz_list,
                    [operator] * self.k,
                    [parameters for n in range(self.k)],
                )
                energy_gradients = job.result().gradients
                weighted_energy_sum_gradient = sum(
                    [self.weight_vector[n] * energy_gradients[n] for n in range(self.k)]
                )
            except Exception as exc:
                raise AlgorithmError(
                    "The primitive job to evaluate the gradient failed!"
                ) from exc

            return weighted_energy_sum_gradient

        return evaluate_gradient

    def _check_circuit_num_qubits(
        self,
        operator: BaseOperator | SparsePauliOp,
        circuit: QuantumCircuit,
        circuit_type: str,
    ) -> QuantumCircuit:
        """Check that the number of qubits for the circuit passed matches
        the number of qubits  of the operator.
        """
        if operator.num_qubits != circuit.num_qubits:
            try:
                logger.info(
                    "Trying to resize %s to match operator on %s qubits.",
                    circuit_type,
                    operator.num_qubits,
                )
                circuit.num_qubits = operator.num_qubits
            except AttributeError as error:
                raise AlgorithmError(
                    f"The number of qubits of the {circuit_type} does not match the ",
                    f"operator, and the {circuit_type} does not allow setting the "
                    "number of qubits using `num_qubits`.",
                ) from error
        return circuit

    def _check_operator_ansatz(
        self, operator: BaseOperator | SparsePauliOp
    ) -> QuantumCircuit:
        """Check that the number of qubits of operator and ansatz match and that the ansatz is
        parameterized.
        """
        # set defaults
        if self.ansatz is None:
            ansatz = RealAmplitudes(num_qubits=operator.num_qubits, reps=6)
        else:
            ansatz = self.ansatz

        ansatz = self._check_circuit_num_qubits(
            operator=operator, circuit=ansatz, circuit_type="ansatz"
        )

        if ansatz.num_parameters == 0:
            raise AlgorithmError(
                "The ansatz must be parameterized, but has no free parameters."
            )

        return ansatz

    def _check_operator_initial_states(
        self,
        list_of_states: list[QuantumCircuit] | None,
        operator: BaseOperator | SparsePauliOp,
    ) -> QuantumCircuit:

        """Check that the number of qubits of operator and all the initial states match."""

        if list_of_states is None:
            initial_states = [
                QuantumCircuit(operator.num_qubits) for n in range(self.k)
            ]
            for n in range(self.k):
                initial_states[n].initialize(
                    Statevector.from_int(n, 2**operator.num_qubits)
                )

            warnings.warn(
                "No initial states have been provided to SSVQE, so they have been set to "
                "a subset of the computational basis states. This may result in unphysical "
                "results for some chemistry and physics problems."
            )

        else:
            initial_states = list_of_states
            if self.check_initial_states_orthogonal is True:
                stacked_states_array = np.hstack(
                    [np.asarray(Statevector(state)) for state in initial_states]
                )
                if not np.isclose(
                    stacked_states_array.transpose() @ stacked_states_array,
                    np.eye(self.k),
                ).any():
                    raise AlgorithmError(
                        "The set of initial states provided is not mutually orthogonal."
                    )

        for initial_state in initial_states:
            initial_state = self._check_circuit_num_qubits(
                operator=operator, circuit=initial_state, circuit_type="initial state"
            )

        return initial_states

    def _check_weight_vector(self, weight_vector: Sequence[float]) -> Sequence[float]:
        """Check that the number of weights matches the number of states."""
        if weight_vector is None:
            weight_vector = [self.k - n for n in range(self.k)]
        elif len(weight_vector) != self.k:
            raise AlgorithmError(
                "The number of weights provided does not match the number of states."
            )

        return weight_vector

    def _eval_aux_ops(
        self,
        ansatz: QuantumCircuit,
        aux_operators: ListOrDict[BaseOperator | SparsePauliOp],
    ) -> ListOrDict[tuple(complex, complex)]:
        """Compute auxiliary operator eigenvalues."""

        if isinstance(aux_operators, dict):
            aux_ops = list(aux_operators.values())
        else:
            aux_ops = aux_operators

        num_aux_ops = len(aux_ops)

        try:
            aux_job = self.estimator.run([ansatz] * num_aux_ops, aux_ops)
            aux_values = aux_job.result().values
            aux_values = list(zip(aux_values, [0] * len(aux_values)))

            if isinstance(aux_operators, dict):
                aux_values = dict(zip(aux_operators.keys(), aux_values))

        except Exception as exc:
            raise AlgorithmError(
                "The primitive job to evaluate the aux operator values failed!"
            ) from exc

        return aux_values

    def _build_ssvqe_result(
        self,
        optimizer_result: OptimizerResult,
        aux_operators_evaluated: ListOrDict[tuple[complex, tuple[complex, int]]],
        optimizer_time: float,
        operator: BaseOperator | SparsePauliOp,
        initialized_ansatz_list: list[QuantumCircuit],
    ) -> SSVQEResult:
        result = SSVQEResult()

        try:
            result.eigenvalues = (
                self.estimator.run(
                    initialized_ansatz_list,
                    [operator] * self.k,
                    [optimizer_result.x] * self.k,
                )
                .result()
                .values
            )

        except Exception as exc:
            raise AlgorithmError(
                "The primitive job to evaluate the eigenvalues failed!"
            ) from exc

        result.cost_function_evals = optimizer_result.nfev
        result.optimal_point = optimizer_result.x
        result.optimal_parameters = dict(
            zip(self.ansatz.parameters, optimizer_result.x)
        )
        result.optimal_value = optimizer_result.fun
        result.optimizer_time = optimizer_time
        result.aux_operators_evaluated = aux_operators_evaluated
        result.optimizer_result = optimizer_result

        return result


class SSVQEResult(VariationalResult, EigensolverResult):
    """SSVQE Result."""

    def __init__(self) -> None:
        super().__init__()
        self._cost_function_evals = None

    @property
    def cost_function_evals(self) -> int:
        """Returns number of cost optimizer evaluations"""
        return self._cost_function_evals

    @cost_function_evals.setter
    def cost_function_evals(self, value: int) -> None:
        """Sets number of cost function evaluations"""
        self._cost_function_evals = value


def _validate_initial_point(point, ansatz):
    expected_size = ansatz.num_parameters

    # try getting the initial point from the ansatz
    if point is None and hasattr(ansatz, "preferred_init_points"):
        point = ansatz.preferred_init_points
    # if the point is None choose a random initial point

    if point is None:
        # get bounds if ansatz has them set, otherwise use [-2pi, 2pi] for each parameter
        bounds = getattr(ansatz, "parameter_bounds", None)
        if bounds is None:
            bounds = [(-2 * np.pi, 2 * np.pi)] * expected_size

        # replace all Nones by [-2pi, 2pi]
        lower_bounds = []
        upper_bounds = []
        for lower, upper in bounds:
            lower_bounds.append(lower if lower is not None else -2 * np.pi)
            upper_bounds.append(upper if upper is not None else 2 * np.pi)

        # sample from within bounds
        point = algorithm_globals.random.uniform(lower_bounds, upper_bounds)

    elif len(point) != expected_size:
        raise ValueError(
            f"The dimension of the initial point ({len(point)}) does not match the "
            f"number of parameters in the circuit ({expected_size})."
        )

    return point


def _validate_bounds(ansatz):
    if hasattr(ansatz, "parameter_bounds") and ansatz.parameter_bounds is not None:
        bounds = ansatz.parameter_bounds
        if len(bounds) != ansatz.num_parameters:
            raise ValueError(
                f"The number of bounds ({len(bounds)}) does not match the number of "
                f"parameters in the circuit ({ansatz.num_parameters})."
            )
    else:
        bounds = [(None, None)] * ansatz.num_parameters

    return bounds

### Constructing the Schlogl operator

In [47]:
def TridiagA(lambda_func, mu_func, N):
    """
    Creates tridiagonal stochastic matrix for the Schlogl model
    Inputs lambda and mu model functions, and N desired size of matrix
    Returns stochastic matrix (non-hermitian)
    """
    # initialize diagonals
    d1 = np.zeros(N - 1)
    d2 = np.zeros(N)
    d3 = np.zeros(N - 1)

    # element 1,1
    d2[0] = -lambda_func(0)

    # element N,N
    d2[N - 1] = -mu_func(N - 1)

    # bottom diagonal elements
    for i in range(N - 1):
        d1[i] = lambda_func(i)

    # top diagonal elements
    for i in range(1, N):
        d3[i - 1] = mu_func(i)

    # main diagonal elements
    for i in range(1, N - 1):
        d2[i] = -lambda_func(i) - mu_func(i)

    # putting the diagonals together
    Q = np.diag(d1, k = -1) + np.diag(d2, k = 0) + np.diag(d3, k = 1)
    
    return Q

def get_volume_array(start_V, stop_V, n_operator_qubits):
    # Function to generate the initial volume array (to carry out computations)
    
    num_elements = 2 ** n_operator_qubits
    step_size = (stop_V - start_V)/(num_elements - 1)
    
    # Generate the volume array, given that the step size has been determined
    volume_array = np.arange(start_V, stop_V, step_size)
    
    return volume_array

def get_unitary(matrix, add_half = False):
    """
    Purpose: given a matrix, returns the unitary, hermitian matrix to be diagonalized
    Input: matrix -> the matrix to be diagonalized
    Output: U -> the unitary matrix
            nqubits -> the number of qubis needed to represent the basis of U
            dimension -> the dimension of the original matrix
    """
    assert matrix.ndim == 2, "Error: Only a matrix maybe processed"
    assert matrix.shape[0] == matrix.shape[1], "Error: Only a square matrix maybe processed"

    if np.any(np.transpose(matrix) != matrix):
        matrix_T = np.transpose(matrix)
        matrix = np.dot(matrix_T, matrix)

    ## Finding the dimension of the matrix
    dimension_hermitian = matrix.shape[0]

    ## Finding the number of qubits required to represent the matrix
    nqubits = int(np.ceil(np.log2(dimension_hermitian)))

    ## Construct the relevant matrix
    op_dim = 2 ** nqubits
    op = np.eye(op_dim)
    op[0:dimension_hermitian, 0:dimension_hermitian] = np.copy(matrix)

    if add_half:
        op = op + np.pi * np.eye(op.shape[0])

    U = la.expm(1j*op)
    
    # Get the dimensions of the unitary matrix
    dimension = U.shape[0]

    return U, nqubits, dimension

def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes

    # Get absolute values
    b = np.abs(b)
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

def compute_rmsd(list1, list2):
    # Ensure the lists have the same length
    
    if len(list1) != len(list2):
        raise ValueError("Lists must have the same length.")
    
    # Compute the RMSD
    rmsd = np.sqrt(np.mean((np.array(list1) - np.array(list2)) ** 2))
    return rmsd

def get_expectation(matrix, zeromode):
    # Compute the expectation value of the Schlogl operator in the steady-state
    
    # Convert the zeromode into a matrix (column vector)    
    zeromode = np.array(zeromode).reshape(len(zeromode), 1)
    zeromode = np.abs(zeromode) # get rid of all (extraneous) negative values (since this is a PDF)
    
    # Compute the steady-state expectation value
    value = np.dot(matrix, zeromode)
    expectation_value = np.dot(zeromode.T, value)
    
    return expectation_value

def compute_eigenvalues(matrix):
    """
    Computes the two eigenvalues with the highest real parts of a general matrix.

    Parameters:
        matrix (np.ndarray): The matrix for which to compute the eigenvalues.

    Returns:
        list: A list containing the two eigenvalues with the highest real parts [eigenvalue1, eigenvalue2].
    """
    # Compute all eigenvalues
    eigenvalues = np.linalg.eigvals(matrix)
    
    # Sort by real part in descending order and select the two with the lowest real parts
    lowest_two_eigenvalues = sorted(eigenvalues, key=lambda x: x.real, reverse=False)[:2]

    # Print
    print('The lowest two eigenvalues are:')
    print(lowest_two_eigenvalues)
    print()
    
    return lowest_two_eigenvalues

def compute_relative_errors(exact_eigenvalues, vqd_eigenvalues):
    """
    Computes the relative errors for the first and second eigenvalues for each optimizer.

    Parameters:
        exact_eigenvalues (list): List of the two lowest exact eigenvalues [lambda_1, lambda_2].
        vqd_eigenvalues (dict): Dictionary where keys are optimizer names and values are lists of the two
                                lowest VQD eigenvalues computed by each optimizer.

    Returns:
        dict: Dictionary where each key is an optimizer name and the value is a tuple of the relative errors
              (error_first, error_second).
    """
    # Extract the exact eigenvalues for easy reference
    exact_first, exact_second = exact_eigenvalues
    
    # Initialize a dictionary to store relative errors for each optimizer
    relative_errors = {}
    
    for optimizer, eigenvalues in vqd_eigenvalues.items():
        vqd_first, vqd_second = eigenvalues  # VQD-computed first and second eigenvalues for this optimizer
        
        # Calculate relative errors for the first and second eigenvalues
        error_first = abs((vqd_first - exact_first) / exact_first) if exact_first != 0 else np.nan
        error_second = abs((vqd_second - exact_second) / exact_second) if exact_second != 0 else np.nan
        
        # Store the errors in the dictionary
        relative_errors[optimizer] = (error_first, error_second)
    
    return relative_errors

def get_eigenvector(A, lambda_min):
    """
    Computes the eigenvector corresponding to the given smallest eigenvalue
    using only the matrix and the eigenvalue.

    Parameters:
        A (ndarray): The square matrix.
        lambda_min (float): The smallest eigenvalue.

    Returns:
        ndarray: The absolute value of the corresponding eigenvector.
    """
    # Compute (A - lambda_min * I)
    M = A - lambda_min * np.eye(A.shape[0])

    # Perform SVD to get the null space
    U, S, Vh = np.linalg.svd(M)

    # The eigenvector is the last column of Vh (corresponding to the smallest singular value)
    eigenvector = Vh[-1, :]

    # Normalize and return the absolute values (optional)
    return np.abs(eigenvector / np.linalg.norm(eigenvector))

### SSVQE implementation

In [50]:
# The following orthogonality function is not needed at the moment (kept anyway if it comes in handy at some point)
# Get initial orthogonal states
def get_orthogonal_initial_states(n):
    """Returns two orthogonal initial state circuits for n qubits."""
    # First state: |0⟩^⊗n (default all zeros)
    zero_state = QuantumCircuit(n)

    # Second state: |1⟩^⊗n (apply X gates to all qubits)
    one_state = QuantumCircuit(n)
    one_state.x(range(n))  # Apply X to all qubits

    return zero_state, one_state

#############################################################################################################
# Define the SSVQE computation with varying ansatz depth
def run_ssvqe(matrix, ansatz, optimizer, seed):
    """
    Function to compute the zeromode and eigenvalues for different optimizers and ansatz depths,
    averaged over multiple independent SSVQE runs.

    Args:
        matrix (np.ndarray): The Hamiltonian matrix.
        ansatz (QuantumCircuit): The ansatz circuit.
        optimizer (Optimizer): Classical optimizer for SSVQE.
        seed (int): Seed value for reproducibility.
        eigenvalues_exact (list): List of exact eigenvalues for reference (classical values).

    Returns:
        zeromode (list): The zeromode statevector obtained from SSVQE.
        eigenvalues_vqd (list): List of eigenvalues obtained from SSVQE.
        num_func_calls (int): Number of function calls made by the optimizer.
    """
    
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Define the qubit Hamiltonian
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)

    # Initial point for the classical optimizer
    seed_value = seed
    np.random.seed(seed_value)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    
    # Initializing the Estimator
    estimator = Estimator()

    # Run the VQE algorithm
    @dataclass
    class SSVQELog:
        values: list = None
        parameters: list = None
        
        def __post_init__(self):
            self.values = []
            self.parameters = []

        # Update function to match the expected arguments
        def update(self, count, parameters, mean, _metadata):
            self.values.append(mean)
            self.parameters.append(parameters)

    log = SSVQELog()

    ssvqe = SSVQE(k=2, estimator = estimator, optimizer = optimizer, ansatz = ansatz, callback = log.update)
    result = ssvqe.compute_eigenvalues(qub_hamiltonian)

    # Get the optimizer runtime (not used in the current context, but can be logged)
    # time = result.optimizer_times

    # Extract the optimal parameters
    optimal_params = result.optimal_point
    final_circuit = ansatz.assign_parameters(optimal_params)
    zeromode_ssvqe = Statevector.from_instruction(final_circuit)
    zeromode = zeromode_ssvqe.data.tolist()

    # Extract the first and second eigenvalues from SSVQE results
    eigenvalues_ssvqe = result.eigenvalues

    # Get the number of function calls
    num_func_calls = result.cost_function_evals

    # Get the gate count and circuit depth of the variational ansatz
    decomposed_ansatz = ansatz.decompose()  
    gate_count_dict = decomposed_ansatz.count_ops()
    gate_count = sum(gate_count_dict.values())
    circuit_depth = decomposed_ansatz.depth()  # Depth of the ansatz circuit

    return zeromode, eigenvalues_ssvqe, gate_count, circuit_depth, num_func_calls

## Get the matrix and the zeromode

In [53]:
def get_zeromode(num_operator_qubits, V):
    # Function to get the matrix and the zeromode

    ## Computing the block diagonal representation of the Schlogl operator matrix
    # Defining parameters
    a = 1
    b = 1
    k1 = 3
    k2 = 0.6
    k3 = 0.25
    k4 = 2.95
    
    # Number of qubits
    num_operator_qubits = num_operator_qubits
    
    # Matrix dimensions
    N = 2 ** num_operator_qubits
    
    # Generating the basis size array
    x_axis = []
    
    for i in range(N):
        x_axis.append(i)
    
    # # Constructing the Schlogl operator for V = 1.1
    # # Get the volume array
    # start_V = 0.1
    # stop_V = 1.6
    # volume_array = get_volume_array(start_V, stop_V, num_operator_qubits)
    
    # # For system volume V = 1.1
    # volume_array = np.arange(0.1, 10.6, 0.1)
    
    # # Construct the matrix representation of the operator
    # for i, V in enumerate(volume_array):
        
    #     # Birth and death rates
    #     lambda_fn = lambda n: ((a*k1*n*(n-1))/V + b*k3*V)
    #     mu_fn = lambda n: ((k2*n*(n-1)*(n-2))/V**2 + n*k4)
    
    #     # stochastic matrix Q of dimension N x N
    #     Q = TridiagA(lambda_fn, mu_fn, N)
        
    #     i += 1
    # ######################################################################  
    # Construct the matrix
    # Birth and death rates
    lambda_fn = lambda n: ((a*k1*n*(n-1))/V + b*k3*V)
    mu_fn = lambda n: ((k2*n*(n-1)*(n-2))/V**2 + n*k4)

    # stochastic matrix Q of dimension N x N
    Q = TridiagA(lambda_fn, mu_fn, N)
    
    # Print the original Schlogl operator matrix
    print('The Schlogl operator matrix is:')
    print(Q)
    print()
        
    # Compute the Hermitian form of the matrix
    hermitian_matrix = np.dot(Q.T, Q)
    
    # # Print the volume array
    # print('The volume array is:')
    # print(volume_array)
    # print()
    
    # Print the volume parameter for which the simulations are being run
    print('The volume parameter is:')
    print(V)
    print()
    
    # Print the Hermitian matrix (block diagonal form)
    print('The Hermitian form of the Schlogl matrix is:')
    print(hermitian_matrix)
    print()
    
   ## Get the classical zeromode
    A, P = la.eig(Q)
    
    A_real = np.real(A)
    index = np.where(A_real == np.amax(A_real))[0][0]
    
    eigenvalue = A[index]
    zeromode_classic = P[:, index]
    
    zeromode_classic = np.real(normalize_eigenvector(zeromode_classic))
    zeromode_classic = np.reshape(zeromode_classic, (zeromode_classic.size, 1))
    
    print("the available eigenvalues are: \n" + str(A))
    print()
    
    print("The minimum eigenvalue located is: \n" + str(eigenvalue))
    print()
    
    print("The minimum zeromode located is: \n" + str(np.real(zeromode_classic)))

    return Q, hermitian_matrix, zeromode_classic

## Determine performance metrics

In [56]:
def performance_metrics(matrix, classical_eigenvalue, pair_depth_dict):
    """
    Estimate VQE resource usage and relative error statistics for given optimizer-ansatz pairs at fixed depths.
    
    Parameters:
    - matrix: Hamiltonian matrix
    - pair_depth_dict: dict of form {'SLSQP-RealAmplitudes': 2, 'P_BFGS-EfficientSU2': 3, ...}
    - classical_eigenvalue: first eigenvalue obtained via exact diagonalization

    Returns:
    - error_stats: dict with mean and std of relative error for each pair
    - resource_info: dict with function call count, gate count, and circuit depth for each pair
    """

    # Get the qubit number 
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Mapping of names to classes
    optimizer_map = {
        "SLSQP": SLSQP,
        "P_BFGS": P_BFGS
    }

    ansatz_map = {
        "RealAmplitudes": RealAmplitudes,
        "TwoLocal": TwoLocal,
        "EfficientSU2": EfficientSU2,
    }

    # Initialize
    eigenvalue_stats = {}

    for pair_name, depth in pair_depth_dict.items():
        print(f"\nRunning SSVQE for: {pair_name} with depth {depth}")

        # Split pair_name to get optimizer and ansatz
        opt_name, ansatz_name = pair_name.split('-')
        optimizer_class = optimizer_map[opt_name]
        AnsatzClass = ansatz_map[ansatz_name]

        # Instantiate optimizer with correct stopping condition
        if opt_name == "P_BFGS":
            optimizer = optimizer_class(maxfun=5000)
        else:
            optimizer = optimizer_class(maxiter=5000)

        # Initialize ansatz
        if AnsatzClass == RealAmplitudes:
            ansatz = AnsatzClass(num_qubits=num_qubits, entanglement='full', reps=depth)
        elif AnsatzClass == TwoLocal:
            ansatz = AnsatzClass(num_qubits=num_qubits, rotation_blocks=['ry'],
                                 entanglement_blocks='cx', reps=depth)
        elif AnsatzClass == EfficientSU2:
            ansatz = AnsatzClass(num_qubits=num_qubits, su2_gates=['ry'],
                                 entanglement='sca', reps=depth)

        # Run 10 independent SSVQE runs
        all_eigenvalues = []
        all_errors = []

        for run in range(10):
            seed = run + 1

            _, eigenvalues_vqd, _, _, _ = run_ssvqe(
                matrix=matrix,
                ansatz=ansatz,
                optimizer=optimizer,
                seed=seed,
            )

            # Get the first eigenvalue and store accordingly; also compute the relative error and store
            quantum_eigenvalue = eigenvalues_vqd[1]
            all_eigenvalues.append(quantum_eigenvalue)
            relative_error = np.abs((quantum_eigenvalue - classical_eigenvalue) / classical_eigenvalue)
            all_errors.append(relative_error)
            
        # Compute stats
        std_eigenvalue = np.std(all_eigenvalues)

        # Determine the lowest relative error in \lambda_1
        lowest_error = np.min(all_errors)

        # Store results
        eigenvalue_stats[pair_name] = {
            'std_eigenvalue': std_eigenvalue, 
            'lowest_error': lowest_error 
        }

        print(f"{pair_name} — Std Dev: {std_eigenvalue:.5f}, Lowest relative error: {lowest_error:.5f}")

    return eigenvalue_stats

## Quantum (SSVQE) analysis

### For V = 1.1 (N = 2)

### Get the matrix and the zeromode

In [20]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(2, 1.1)

## Get the classical eigenvalues
classical_eigenvalues = compute_eigenvalues(hermitian_matrix)
second_eigenvalue = classical_eigenvalues[1]

The Schlogl operator matrix is:
[[ -0.275        2.95         0.           0.        ]
 [  0.275       -3.225        5.9          0.        ]
 [  0.           0.275      -11.62954545  11.82520661]
 [  0.           0.           5.72954545 -11.82520661]]

The volume parameter is:
1.1

The Hermitian form of the Schlogl matrix is:
[[ 1.51250000e-01 -1.69812500e+00  1.62250000e+00  0.00000000e+00]
 [-1.69812500e+00  1.91787500e+01 -2.22256250e+01  3.25193182e+00]
 [ 1.62250000e+00 -2.22256250e+01  2.02884019e+02 -2.05274837e+02]
 [ 0.00000000e+00  3.25193182e+00 -2.05274837e+02  2.79671023e+02]]

the available eigenvalues are: 
[  0.        +0.j  -2.60510311+0.j  -4.34244163+0.j -20.00720732+0.j]

The minimum eigenvalue located is: 
0j

The minimum zeromode located is: 
[[0.99567159]
 [0.09281684]
 [0.00432621]
 [0.00209613]]
The lowest two eigenvalues are:
[1.0754013022257892e-14, 9.002123036682548]



### Estimate the standard deviation in results / lowest relative errors (fix at 5000 optimization iterations)

In [35]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 2,
    'SLSQP-TwoLocal': 2,
    'SLSQP-EfficientSU2': 2, 
    'P_BFGS-RealAmplitudes': 2, 
    'P_BFGS-TwoLocal': 2,
    'P_BFGS-EfficientSU2': 2
}

# Run SSVQE
std_eigenvalues_N_2 = performance_metrics(hermitian_matrix, second_eigenvalue, pair_depth_dict)


Running SSVQE for: SLSQP-RealAmplitudes with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphys

SLSQP-RealAmplitudes — Std Dev: 5.37181, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-TwoLocal with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided

SLSQP-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-EfficientSU2 with depth 2


...

P_BFGS-RealAmplitudes — Std Dev: 7.33710, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-TwoLocal with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-EfficientSU2 with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-EfficientSU2 — Std Dev: 10.46175, Lowest relative error: 0.00000


### Print performance metrics

In [38]:
## Print performance metrics
print('The performance metrics are:')
print(std_eigenvalues_N_2)
print()

The performance metrics are:
{'SLSQP-RealAmplitudes': {'std_eigenvalue': 5.371809616209936, 'lowest_error': 1.9784838238919477e-09}, 'SLSQP-TwoLocal': {'std_eigenvalue': 8.311242930688961e-08, 'lowest_error': 2.839407019559818e-10}, 'SLSQP-EfficientSU2': {'std_eigenvalue': 86.47153729394253, 'lowest_error': 7.969493737086103e-10}, 'P_BFGS-RealAmplitudes': {'std_eigenvalue': 7.337095138039193, 'lowest_error': 2.468553692317278e-13}, 'P_BFGS-TwoLocal': {'std_eigenvalue': 1.0016715402083474e-12, 'lowest_error': 9.866321711899594e-16}, 'P_BFGS-EfficientSU2': {'std_eigenvalue': 10.461745017585017, 'lowest_error': 4.0451919018788334e-14}}



### For V = 1.1 (N = 3)

### Get the matrix and the zeromode

In [23]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(3, 1.1)

## Get the classical eigenvalues
classical_eigenvalues = compute_eigenvalues(hermitian_matrix)
second_eigenvalue = classical_eigenvalues[1]

The Schlogl operator matrix is:
[[  -0.275         2.95          0.            0.            0.
     0.            0.            0.        ]
 [   0.275        -3.225         5.9           0.            0.
     0.            0.            0.        ]
 [   0.            0.275       -11.62954545   11.82520661    0.
     0.            0.            0.        ]
 [   0.            0.            5.72954545  -28.46384298   23.70082645
     0.            0.            0.        ]
 [   0.            0.            0.           16.63863636  -56.70309917
    44.50206612    0.            0.        ]
 [   0.            0.            0.            0.           33.00227273
   -99.32252066   77.20413223    0.        ]
 [   0.            0.            0.            0.            0.
    54.82045455 -159.29731405  124.7822314 ]
 [   0.            0.            0.            0.            0.
     0.           82.09318182 -124.7822314 ]]

The volume parameter is:
1.1

The Hermitian form of the Schlogl matrix

### Estimating the standard deviation in results / lowest relative errors (fix at 5000 optimization iterations)

In [26]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 4,
    'SLSQP-TwoLocal': 4,
    'SLSQP-EfficientSU2': 6, 
    'P_BFGS-RealAmplitudes': 4, 
    'P_BFGS-TwoLocal': 4,
    'P_BFGS-EfficientSU2': 7
}

# Run SSVQE
std_eigenvalues_N_3 = performance_metrics(hermitian_matrix, second_eigenvalue, pair_depth_dict)


Running SSVQE for: SLSQP-RealAmplitudes with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\AppData\Local\Temp\

SLSQP-RealAmplitudes — Std Dev: 0.20793, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-TwoLocal with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided

SLSQP-TwoLocal — Std Dev: 0.17982, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-EfficientSU2 with depth 6


...

P_BFGS-RealAmplitudes — Std Dev: 2.84813, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-TwoLocal with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-TwoLocal — Std Dev: 3.08679, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-EfficientSU2 with depth 7


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-EfficientSU2 — Std Dev: 0.65501, Lowest relative error: 0.00000


### Print performance metrics

In [29]:
## Print performance metrics
print('The performance metrics are:')
print(std_eigenvalues_N_3)
print()

The performance metrics are:
{'SLSQP-RealAmplitudes': {'std_eigenvalue': 0.20793347056308673, 'lowest_error': 2.5320310212813057e-09}, 'SLSQP-TwoLocal': {'std_eigenvalue': 0.17981675060892915, 'lowest_error': 2.1480336619167836e-10}, 'SLSQP-EfficientSU2': {'std_eigenvalue': 0.0002651470675921751, 'lowest_error': 4.2820135093252804e-08}, 'P_BFGS-RealAmplitudes': {'std_eigenvalue': 2.8481348514985396, 'lowest_error': 6.664454300754493e-08}, 'P_BFGS-TwoLocal': {'std_eigenvalue': 3.0867945313581657, 'lowest_error': 1.359828369288348e-07}, 'P_BFGS-EfficientSU2': {'std_eigenvalue': 0.6550109586125017, 'lowest_error': 2.031284499404519e-06}}



### For V = 10.5 (N = 2)

### Get the matrix and the zeromode

In [41]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(2, 10.5)

## Get the classical eigenvalues
classical_eigenvalues = compute_eigenvalues(hermitian_matrix)
second_eigenvalue = classical_eigenvalues[1]

The Schlogl operator matrix is:
[[-2.625       2.95        0.          0.        ]
 [ 2.625      -5.575       5.9         0.        ]
 [ 0.          2.625      -9.09642857  8.88265306]
 [ 0.          0.          3.19642857 -8.88265306]]

The volume parameter is:
10.5

The Hermitian form of the Schlogl matrix is:
[[  13.78125     -22.378125     15.4875        0.        ]
 [ -22.378125     46.67375     -56.770625     23.31696429]
 [  15.4875      -56.770625    127.77216837 -109.19318513]
 [   0.           23.31696429 -109.19318513  157.80305081]]

the available eigenvalues are: 
[-1.52577371e+01+0.j -7.74424342e+00+0.j -9.70174270e-16+0.j
 -3.17710106e+00+0.j]

The minimum eigenvalue located is: 
(-9.701742699678762e-16+0j)

The minimum zeromode located is: 
[[-0.71268205]
 [-0.63416623]
 [-0.28215023]
 [-0.10153195]]
The lowest two eigenvalues are:
[-4.3517946982977e-15, 10.364814601313393]



### Estimate the standard deviation in results / lowest relative errors (fix at 5000 optimization iterations)

In [44]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 2,
    'SLSQP-TwoLocal': 2,
    'SLSQP-EfficientSU2': 2, 
    'P_BFGS-RealAmplitudes': 2, 
    'P_BFGS-TwoLocal': 2,
    'P_BFGS-EfficientSU2': 2
}

# Run SSVQE
std_eigenvalues_N_2 = performance_metrics(hermitian_matrix, second_eigenvalue, pair_depth_dict)


Running SSVQE for: SLSQP-RealAmplitudes with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided

SLSQP-RealAmplitudes — Std Dev: 16.41777, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-TwoLocal with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided

SLSQP-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-EfficientSU2 with depth 2


...

P_BFGS-RealAmplitudes — Std Dev: 16.44468, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-TwoLocal with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-EfficientSU2 with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_36592\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-EfficientSU2 — Std Dev: 11.73390, Lowest relative error: 0.00000


### Print performance metrics 

In [46]:
## Print performance metrics
print('The performance metrics are:')
print(std_eigenvalues_N_2)
print()

The performance metrics are:
{'SLSQP-RealAmplitudes': {'std_eigenvalue': 16.41776583458519, 'lowest_error': 1.16682424569862e-10}, 'SLSQP-TwoLocal': {'std_eigenvalue': 9.855303726772845e-08, 'lowest_error': 3.544002398840326e-11}, 'SLSQP-EfficientSU2': {'std_eigenvalue': 11.067279573048227, 'lowest_error': 1.4355823311671692e-09}, 'P_BFGS-RealAmplitudes': {'std_eigenvalue': 16.44467828417353, 'lowest_error': 4.455967578865721e-15}, 'P_BFGS-TwoLocal': {'std_eigenvalue': 3.9147052016470815e-13, 'lowest_error': 6.855334736716493e-16}, 'P_BFGS-EfficientSU2': {'std_eigenvalue': 11.733899833830806, 'lowest_error': 8.397785052477705e-15}}



### For V = 10.5 (N = 3)

### Get the matrix and the zeromode

In [32]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(3, 10.5)

## Get the classical eigenvalues
classical_eigenvalues = compute_eigenvalues(hermitian_matrix)
second_eigenvalue = classical_eigenvalues[1]

The Schlogl operator matrix is:
[[ -2.625        2.95         0.           0.           0.
    0.           0.           0.        ]
 [  2.625       -5.575        5.9          0.           0.
    0.           0.           0.        ]
 [  0.           2.625       -9.09642857   8.88265306   0.
    0.           0.           0.        ]
 [  0.           0.           3.19642857 -13.22193878  11.93061224
    0.           0.           0.        ]
 [  0.           0.           0.           4.33928571 -17.98418367
   15.07653061   0.           0.        ]
 [  0.           0.           0.           0.           6.05357143
  -23.41581633  18.35306122   0.        ]
 [  0.           0.           0.           0.           0.
    8.33928571 -29.5494898   21.79285714]
 [  0.           0.           0.           0.           0.
    0.          11.19642857 -21.79285714]]

The volume parameter is:
10.5

The Hermitian form of the Schlogl matrix is:
[[  13.78125     -22.378125     15.4875        0.         

### Estimating the standard deviation in results / lowest relative errors (fix at 5000 optimization iterations)

In [35]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 4,
    'SLSQP-TwoLocal': 4,
    'SLSQP-EfficientSU2': 6, 
    'P_BFGS-RealAmplitudes': 4, 
    'P_BFGS-TwoLocal': 4,
    'P_BFGS-EfficientSU2': 6
}

# Run SSVQE
std_eigenvalues_N_3 = performance_metrics(hermitian_matrix, second_eigenvalue, pair_depth_dict)


Running SSVQE for: SLSQP-RealAmplitudes with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\AppData\Local\Temp\

SLSQP-RealAmplitudes — Std Dev: 1.78000, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-TwoLocal with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided

SLSQP-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-EfficientSU2 with depth 6


...

P_BFGS-RealAmplitudes — Std Dev: 0.69758, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-TwoLocal with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-EfficientSU2 with depth 6


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-EfficientSU2 — Std Dev: 15.73502, Lowest relative error: 0.00000


### Print performance metrics

In [38]:
## Print performance metrics
print('The performance metrics are:')
print(std_eigenvalues_N_3)
print()

The performance metrics are:
{'SLSQP-RealAmplitudes': {'std_eigenvalue': 1.779998395735703, 'lowest_error': 1.449288687033166e-09}, 'SLSQP-TwoLocal': {'std_eigenvalue': 1.65054182264871e-07, 'lowest_error': 1.9118397080813554e-10}, 'SLSQP-EfficientSU2': {'std_eigenvalue': 7.40795546309687, 'lowest_error': 4.448400499621328e-09}, 'P_BFGS-RealAmplitudes': {'std_eigenvalue': 0.6975787827807138, 'lowest_error': 1.1256635387765011e-12}, 'P_BFGS-TwoLocal': {'std_eigenvalue': 5.09033482589815e-11, 'lowest_error': 8.594967906914172e-14}, 'P_BFGS-EfficientSU2': {'std_eigenvalue': 15.735016528321799, 'lowest_error': 1.940799204787071e-14}}



### Also, run simulations for V = 5.5 for better clarity

### For V = 5.5 (N = 2) --- get the matrix and the zeromode

In [60]:
## Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(2, 5.5)

## Get the classical eigenvalues
classical_eigenvalues = compute_eigenvalues(hermitian_matrix)
second_eigenvalue = classical_eigenvalues[1]

The Schlogl operator matrix is:
[[-1.375       2.95        0.          0.        ]
 [ 1.375      -4.325       5.9         0.        ]
 [ 0.          1.375      -8.36590909  8.96900826]
 [ 0.          0.          2.46590909 -8.96900826]]

The volume parameter is:
5.5

The Hermitian form of the Schlogl matrix is:
[[  3.78125    -10.003125     8.1125       0.        ]
 [-10.003125    29.29875    -37.020625    12.33238636]
 [  8.1125     -37.020625   110.87914256 -97.15066679]
 [  0.          12.33238636 -97.15066679 160.8862185 ]]

the available eigenvalues are: 
[ 4.44089210e-16+0.j -2.84354832e+00+0.j -6.37631943e+00+0.j
 -1.38150496e+01+0.j]

The minimum eigenvalue located is: 
(4.440892098500626e-16+0j)

The minimum zeromode located is: 
[[-0.90169082]
 [-0.42027962]
 [-0.09794652]
 [-0.02692909]]
The lowest two eigenvalues are:
[-4.257371493713317e-15, 9.1459297986593]



In [62]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 2,
    'SLSQP-TwoLocal': 2,
    'SLSQP-EfficientSU2': 2, 
    'P_BFGS-RealAmplitudes': 2, 
    'P_BFGS-TwoLocal': 2,
    'P_BFGS-EfficientSU2': 2
}

# Run SSVQE
std_eigenvalues_N_2 = performance_metrics(hermitian_matrix, second_eigenvalue, pair_depth_dict)


Running SSVQE for: SLSQP-RealAmplitudes with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphys

SLSQP-RealAmplitudes — Std Dev: 8.87563, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-TwoLocal with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided

SLSQP-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-EfficientSU2 with depth 2


...

P_BFGS-RealAmplitudes — Std Dev: 10.29852, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-TwoLocal with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-EfficientSU2 with depth 2


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-EfficientSU2 — Std Dev: 29.09452, Lowest relative error: 0.00000


### Print performance metrics

In [65]:
## Print performance metrics
print('The performance metrics are:')
print(std_eigenvalues_N_2)
print()

The performance metrics are:
{'SLSQP-RealAmplitudes': {'std_eigenvalue': 8.87563021438117, 'lowest_error': 9.737272060255562e-11}, 'SLSQP-TwoLocal': {'std_eigenvalue': 1.77488895461356e-07, 'lowest_error': 4.259942662684739e-10}, 'SLSQP-EfficientSU2': {'std_eigenvalue': 12.39823544397481, 'lowest_error': 4.933370808067898e-10}, 'P_BFGS-RealAmplitudes': {'std_eigenvalue': 10.298520280657305, 'lowest_error': 7.768950247838756e-16}, 'P_BFGS-TwoLocal': {'std_eigenvalue': 1.1337618923054265e-12, 'lowest_error': 1.942237561959689e-16}, 'P_BFGS-EfficientSU2': {'std_eigenvalue': 29.094516248982472, 'lowest_error': 2.5637535817867892e-14}}



### For V = 5.5 (N = 3)

### Get the matrix and the zeromode

In [68]:
# Matrix and zeromode
Q, hermitian_matrix, zeromode = get_zeromode(3, 5.5)

## Get the classical eigenvalues
classical_eigenvalues = compute_eigenvalues(hermitian_matrix)
second_eigenvalue = classical_eigenvalues[1]

The Schlogl operator matrix is:
[[ -1.375        2.95         0.           0.           0.
    0.           0.           0.        ]
 [  1.375       -4.325        5.9          0.           0.
    0.           0.           0.        ]
 [  0.           1.375       -8.36590909   8.96900826   0.
    0.           0.           0.        ]
 [  0.           0.           2.46590909 -13.61673554  12.27603306
    0.           0.           0.        ]
 [  0.           0.           0.           4.64772727 -20.1964876
   15.94008264   0.           0.        ]
 [  0.           0.           0.           0.           7.92045455
  -28.22417355  20.08016529   0.        ]
 [  0.           0.           0.           0.           0.
   12.28409091 -37.81880165  24.81528926]
 [  0.           0.           0.           0.           0.
    0.          17.73863636 -24.81528926]]

The volume parameter is:
5.5

The Hermitian form of the Schlogl matrix is:
[[    3.78125      -10.003125       8.1125         0.
      

### Estimating the standard deviation in results / lowest relative errors (fix at 5000 optimization iterations)

In [71]:
# Initialize the optimizer -- ansatz pairings and corresponding depths
pair_depth_dict = {
    'SLSQP-RealAmplitudes': 4,
    'SLSQP-TwoLocal': 4,
    'SLSQP-EfficientSU2': 6, 
    'P_BFGS-RealAmplitudes': 4, 
    'P_BFGS-TwoLocal': 4,
    'P_BFGS-EfficientSU2': 6
}

# Run SSVQE
std_eigenvalues_N_3 = performance_metrics(hermitian_matrix, second_eigenvalue, pair_depth_dict)


Running SSVQE for: SLSQP-RealAmplitudes with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\anaconda3\Lib\site-packages\scipy\optimize\_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
C:\Users\Owner\AppData\Local\Temp\

SLSQP-RealAmplitudes — Std Dev: 0.19208, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-TwoLocal with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided

SLSQP-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: SLSQP-EfficientSU2 with depth 6


...

P_BFGS-RealAmplitudes — Std Dev: 1.30335, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-TwoLocal with depth 4


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-TwoLocal — Std Dev: 0.00000, Lowest relative error: 0.00000

Running SSVQE for: P_BFGS-EfficientSU2 with depth 6


C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physics problems.
  warnings.warn(
For Windows, using only current process. Multiple core use not supported.
C:\Users\Owner\AppData\Local\Temp\ipykernel_40156\1962045933.py:451: UserWarning: No initial states have been provided to SSVQE, so they have been set to a subset of the computational basis states. This may result in unphysical results for some chemistry and physi

P_BFGS-EfficientSU2 — Std Dev: 1.87544, Lowest relative error: 0.00000


### Print performance metrics

In [74]:
## Print performance metrics
print('The performance metrics are:')
print(std_eigenvalues_N_3)
print()

The performance metrics are:
{'SLSQP-RealAmplitudes': {'std_eigenvalue': 0.19207610062396255, 'lowest_error': 2.708055095150044e-09}, 'SLSQP-TwoLocal': {'std_eigenvalue': 1.0682070433048478e-07, 'lowest_error': 3.790233214736505e-09}, 'SLSQP-EfficientSU2': {'std_eigenvalue': 0.0012452638020385423, 'lowest_error': 1.4335712361156271e-08}, 'P_BFGS-RealAmplitudes': {'std_eigenvalue': 1.3033543328194304, 'lowest_error': 9.263106047356863e-12}, 'P_BFGS-TwoLocal': {'std_eigenvalue': 2.024728656508871e-10, 'lowest_error': 2.976719144871632e-12}, 'P_BFGS-EfficientSU2': {'std_eigenvalue': 1.8754401111775647, 'lowest_error': 2.1073533227120518e-12}}

